In [1]:
import tensorflow as tf

import numpy as np

import pylab as pl
%pylab inline

import pandas as pd
from pandas import Series, DataFrame

Populating the interactive namespace from numpy and matplotlib


Default graph

In [2]:
default_graph = tf.get_default_graph()
print(default_graph)

In [3]:
## Let's see what it contains
default_graph.get_operations()

[]

Whatever operations / tensors you create are added to the default graph:

In [4]:
a = tf.constant(2.0, name = "a")

In [5]:
a.graph

In [6]:
## Let's see what the default graph contains
for op in default_graph.get_operations():
    print(op)

name: "a"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 2.0
    }
  }
}



You can create another Graph and use it as default using a with-block:

In [7]:
g = tf.Graph()
print(g)

In [8]:
with g.as_default():
    print(tf.get_default_graph())
    b = tf.Variable(2.0, name = "b")

In [9]:
## Let's see what g contains
g.get_operations()

[<tf.Operation 'b/initial_value' type=Const>,
 <tf.Operation 'b' type=VariableV2>,
 <tf.Operation 'b/Assign' type=Assign>,
 <tf.Operation 'b/read' type=Identity>]

Create Session

In [10]:
sess = tf.Session()

Run constant `a` in a session

In [11]:
sess.run(a)

2.0

Session remembers a lot of data. After using it, it is good to cloese it explicitely:

In [12]:
sess.close()

A standard practise is tu run a sesion in a with-block so that it is closed automatically.

In [13]:
with tf.Session() as sess:
    print(sess.run(a))

2.0


### Runing a variable from the default graph in a session 

In [14]:
## Add a variable `b1` to the dafault graph
b1 = tf.Variable(initial_value=5.1, name = "b1")

The attempt to evaluate `b1` will produce an error. The variables of the graph must be initialised to the given 'initial_value' in that session.

In [15]:
## this will produce an error
with tf.Session() as sess:
    print(sess.run(b1))

FailedPreconditionError: Attempting to use uninitialized value b1
	 [[Node: _retval_b1_0_0 = _Retval[T=DT_FLOAT, index=0, _device="/job:localhost/replica:0/task:0/cpu:0"](b1)]]

It works fine however if you don't rely on the initial values of your variables and specify them explicitely when running the node.

In [16]:
with tf.Session() as sess:
    print(sess.run(b1, feed_dict={b1 : 4}))

4.0


In order to use the initial values, we need to create a new operation in the graph that, when run in a session, initializes the values of the variables in that session for the that graph.

In [17]:
dg_init = tf.global_variables_initializer()

In [18]:
## our initializer has been added to the defalut_graph:
default_graph.get_operations()

[<tf.Operation 'a' type=Const>,
 <tf.Operation 'b1/initial_value' type=Const>,
 <tf.Operation 'b1' type=VariableV2>,
 <tf.Operation 'b1/Assign' type=Assign>,
 <tf.Operation 'b1/read' type=Identity>,
 <tf.Operation 'init' type=NoOp>]

So in a session we now first run the initializer node and only then we can expect the variables to have the given initial values.

In [19]:
with tf.Session() as sess:
    sess.run(dg_init)
    print(sess.run(b1))

5.1


### Runing a variable from a non-default graph in a session 

In [20]:
## this will produce an error
with tf.Session() as sess:
    print(sess.run(b, feed_dict={b : 4}))

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("b:0", shape=(), dtype=float32_ref) is not an element of this graph.

You must say to TF to either use your graph as the the default:

In [21]:
with g.as_default():
    with tf.Session() as sess:
        print(sess.run(b, feed_dict={b : 3}))

3.0


Or you can pass it to the session:

In [22]:
with tf.Session(graph = g) as sess:
    print(sess.run(b, feed_dict={b : 2}))

2.0


### Mathematical operations
It seems that whenever we do a mathematical oberation between things in a graph, a new node is automatically created and added to our graph.

In [23]:
expr1 = a * b1

In [24]:
expr1

<tf.Tensor 'mul:0' shape=() dtype=float32>

In [25]:
default_graph.get_operations()

[<tf.Operation 'a' type=Const>,
 <tf.Operation 'b1/initial_value' type=Const>,
 <tf.Operation 'b1' type=VariableV2>,
 <tf.Operation 'b1/Assign' type=Assign>,
 <tf.Operation 'b1/read' type=Identity>,
 <tf.Operation 'init' type=NoOp>,
 <tf.Operation 'mul' type=Mul>]

Naturally, a mathematical operation between two nodes from different graphs will result in an error.

In [26]:
## This will produce an error
a * b

ValueError: Tensor("b:0", shape=(), dtype=float32_ref) must be from the same graph as Tensor("a:0", shape=(), dtype=float32).

To evaluate an expression you can either pass the values of all the variables:

In [27]:
with tf.Session() as sess:
    print(sess.run(expr1, feed_dict={b1 : 4}))

8.0


Or you can initialize the variables with their `initial_value`:

In [28]:
with tf.Session() as sess:
    sess.run(dg_init)
    print(sess.run(expr1))

10.2


## Placeholders

In [29]:
c = tf.placeholder("float", name = "c")

In [30]:
default_graph.get_operations()

[<tf.Operation 'a' type=Const>,
 <tf.Operation 'b1/initial_value' type=Const>,
 <tf.Operation 'b1' type=VariableV2>,
 <tf.Operation 'b1/Assign' type=Assign>,
 <tf.Operation 'b1/read' type=Identity>,
 <tf.Operation 'init' type=NoOp>,
 <tf.Operation 'mul' type=Mul>,
 <tf.Operation 'c' type=Placeholder>]

At the moment I don't understand the difference between variables and placeholders.

## Visualization

In [31]:
tb_dir = "tmp/basic3"

with tf.Session() as session:
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter(tb_dir, session.graph)
    writer.add_graph(sess.graph)

Now you 
* run the command-prompt for this winpython. 
* in the command prompt put `tensorboard --logdir=`path/to/log-directory
* in a browser open `localhost:6006`

In [ ]:
## you can run tensorboard here, but it will freeze the kernel
!tensorboard --logdir=tmp/basic3